# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

### Общая информация

Дата выдачи: 28.01.2022

Мягкий дедлайн: 23:59MSK 14.02.2022

Жесткий дедлайн: 23:59MSK 17.02.2022

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимальная оценка за работу (без учёта бонусов) — 10 баллов.

Сдавать задание после указанного жёсткого срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

### Формат сдачи
Задания сдаются через систему anytask. Посылка должна содержать:
* Ноутбук homework-practice-08-random-features-Username.ipynb

Username — ваша фамилия и имя на латинице именно в таком порядке

### О задании

На занятиях мы подробно обсуждали метод опорных векторов (SVM). В базовой версии в нём нет чего-то особенного — мы всего лишь используем специальную функцию потерь, которая не требует устремлять отступы к бесконечности; ей достаточно, чтобы отступы были не меньше +1. Затем мы узнали, что SVM можно переписать в двойственном виде, который, позволяет заменить скалярные произведения объектов на ядра. Это будет соответствовать построению модели в новом пространстве более высокой размерности, координаты которого представляют собой нелинейные модификации исходных признаков.

Ядровой SVM, к сожалению, довольно затратен по памяти (нужно хранить матрицу Грама размера $d \times d$) и по времени (нужно решать задачу условной оптимизации с квадратичной функцией, а это не очень быстро). Мы обсуждали, что есть способы посчитать новые признаки $\tilde \varphi(x)$ на основе исходных так, что скалярные произведения этих новых $\langle \tilde \varphi(x), \tilde \varphi(z) \rangle$ приближают ядро $K(x, z)$.

Мы будем исследовать аппроксимации методом Random Fourier Features (RFF, также в литературе встречается название Random Kitchen Sinks) для гауссовых ядер. Будем использовать формулы, которые немного отличаются от того, что было на лекциях (мы добавим сдвиги внутрь тригонометрических функций и будем использовать только косинусы, потому что с нужным сдвигом косинус превратится в синус):
$$\tilde \varphi(x) = (
\cos (w_1^T x + b_1),
\dots,
\cos (w_n^T x + b_n)
),$$
где $w_j \sim \mathcal{N}(0, 1/\sigma^2)$, $b_j \sim U[-\pi, \pi]$.

На новых признаках $\tilde \varphi(x)$ мы будем строить любую линейную модель.

Можно считать, что это некоторая новая парадигма построения сложных моделей. Можно направленно искать сложные нелинейные закономерности в данных с помощью градиентного бустинга или нейронных сетей, а можно просто нагенерировать большое количество случайных нелинейных признаков и надеяться, что быстрая и простая модель (то есть линейная) сможет показать на них хорошее качество. В этом задании мы изучим, насколько работоспособна такая идея.

### Алгоритм

Вам потребуется реализовать следующий алгоритм:
1. Понизить размерность выборки до new_dim с помощью метода главных компонент.
2. Для полученной выборки оценить гиперпараметр $\sigma^2$ с помощью эвристики (рекомендуем считать медиану не по всем парам объектов, а по случайному подмножеству из где-то миллиона пар объектов): $$\sigma^2 = \text{median}_{i, j = 1, \dots, \ell, i \neq j} \left\{\sum_{k = 1}^{d} (x_{ik} - x_{jk})^2 \right\}$$
3. Сгенерировать n_features наборов весов $w_j$ и сдвигов $b_j$.
4. Сформировать n_features новых признаков по формулам, приведённым выше.
5. Обучить линейную модель (логистическую регрессию или SVM) на новых признаках.
6. Повторить преобразования (PCA, формирование новых признаков) к тестовой выборке и применить модель.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [1]:
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0], -1)
x_test = x_test_pics.reshape(x_test_pics.shape[0], -1)

In [2]:
x_train.shape

(60000, 784)

__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса ниже или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [3]:
import numpy as np

In [5]:
mask = np.random.randint(x_train.shape[0], size=(2, 1000000))
xi, xj = x_train[mask[0]], x_train[mask[1]]

sigma_male = np.median(np.sum((xi - xj)**2, axis=1))
sigma_male

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

54406.0

In [17]:
x_train[mask[0]].shape

(1000000, 784)

In [57]:
w_samples = np.random.normal(0, 1/sigma_male, (1000, x_train.shape[1]))
b = np.random.uniform(-np.pi, np.pi, 1000)

xtrain_newy = np.cos(x_train @ w_samples.T + b)
xtest_newy = np.cos(x_test @ w_samples.T + b)

In [60]:
w_samples.T.shape

(784, 1000)

In [59]:
x_test.shape

(10000, 784)

In [46]:
scaler = StandardScaler()
x_train_newsc = scaler.fit_transform(xtrain_newy)
x_test_newsc = scaler.transform(xtest_newy)

In [24]:
from sklearn.svm import SVC

svc = SVC(kernel='linear', probability=True)
svc.fit(xtrain_newy[:2000], y_train[:2000])
ys_pred = svc.predict(xtest_newy)

In [26]:
svc.predict_proba(xtest_newy).shape

(10000, 10)

In [23]:
# Scaled data, RFF, SVM(linear kernel)
(ys_pred == y_test).mean()

0.7989

In [21]:
# Scaled data, RFF, SVM(rbf kernel)
(ys_pred == y_test).mean()

0.2126

In [14]:
# Scaled data, SVM(rbf kernel)
(ys_pred == y_test).mean()

0.814

In [55]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(solver='saga')
logreg.fit(x_train_newsc[:2000], y_train[:2000])
ys_pred = logreg.predict(x_test_newsc)

C:\Users\Zhan\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [56]:
# Scaled RFF, RFF, logreg(saga)
(ys_pred == y_test).mean()

0.7928

In [48]:
# Scaled RFF, RFF, logreg
(ys_pred == y_test).mean()

0.7852

In [42]:
# Scaled data, Scaled RFF, RFF, logreg
(ys_pred == y_test).mean()

0.7858

In [33]:
# Scaled data, RFF, logreg
(ys_pred == y_test).mean()

0.7631

In [55]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC

class RFFPipeline(BaseEstimator, TransformerMixin):
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, use_scaler=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        new_dim, int: PCA output size.
        
        use_PCA, bool: whether to include PCA preprocessing.
        
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.new_dim = new_dim
        self.classifier = classifier
        self.use_scaler = use_scaler
        
    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        # Sigma squared Male Rule estimation
        mask = np.random.randint(X.shape[0], size=(2, 1000000))
        xi, xj = X[mask[0]], X[mask[1]]
        sigma_sq = np.median(np.sum(((xi - xj) ** 2), axis=1))
        
        # Scaling and PCA
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        if self.use_PCA:
            self.pca = PCA(n_components=self.new_dim)
            X = pca.fit_transform(X)
        
        # Generating features with RFF
        self.w_samples = np.random.normal(0, 1/sigma_sq, (self.n_features, X.shape[1]))
        self.b = np.random.uniform(-np.pi, np.pi, self.n_features)
        Phi_X = np.cos(X @ self.w_samples.T + self.b)
        
        # Scaling new features and fit
        if self.use_scaler:
            self.scaler = StandardScaler()
            Phi_X = self.scaler.fit_transform(Phi_X)
            
        if classifier == 'svm':
            self.svc = SVC(kernel='linear', probability=True)
            self.svc.fit(Phi_X, y)
            
        if classifier == 'logreg':
            self.logreg = LogisticRegression(solver='saga')
            self.logreg.fit(Phi_X, y)
            
        raise NotImplementedError
        return self

    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        # Scaling and PCA
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        if self.use_PCA:
            pca = PCA(n_components=self.new_dim)
            X = pca.fit_transform(X)
        
        # Generating features with RFF
        Phi_X = np.cos(X @ self.w_samples.T + self.b)
        
        # Probabilities
        if classifier == 'svm':
            return self.svc.predict_proba(Phi_X)
            
        if classifier == 'logreg':
            return self.logreg.predict_proba(Phi_X)
        
        raise NotImplementedError
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        # Scaling and PCA
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        if self.use_PCA:
            pca = PCA(n_components=self.new_dim)
            X = pca.fit_transform(X)
            
        # Generating features with RFF
        Phi_X = np.cos(X @ self.w_samples.T + self.b)
        
        # Classification
        if classifier == 'svm':
            return self.svc.predict(Phi_X)
            
        if classifier == 'logreg':
            return self.logreg.predict(Phi_X)
            
        raise NotImplementedError

__Задание 2. (3 балла)__

Сравните подход со случайными признаками с обучением SVM на исходных признаках. Попробуйте вариант с обычным (линейным) SVM и с ядровым SVM. Ядровой SVM может очень долго обучаться, поэтому можно делать любые разумные вещи для ускорения: брать подмножество объектов из обучающей выборки, например.

Сравните подход со случайными признаками с вариантом, в котором вы понижаете размерность с помощью PCA и обучаете градиентный бустинг. Используйте одну из реализаций CatBoost/LightGBM/XGBoost, не забудьте подобрать число деревьев и длину шага.

Сделайте выводы — насколько идея со случайными признаками работает? Сравните как с точки зрения качества, так и с точки зрения скорости обучения и применения.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 3. (2 балла)__

Проведите эксперименты:
1. Помогает ли предварительное понижение размерности с помощью PCA? 
2. Как зависит итоговое качество от n_features? Выходит ли оно на плато при росте n_features?
3. Важно ли, какую модель обучать — логистическую регрессию или SVM?

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

### Бонус

__Задание 4. (Максимум 2 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет). Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪

__Задание 5. (Максимум 2 балла)__

Поэкспериментируйте с функциями для вычисления новых случайных признаков. Не обязательно использовать косинус от скалярного произведения — можно брать знак от него, хэш и т.д. Придумайте побольше вариантов для генерации признаков и проверьте, не получается ли с их помощью добиваться более высокого качества. Также можете попробовать другой классификатор поверх случайных признаков, сравните результаты.

In [ ]:
# Your code here: (￣▽￣)/♫•*¨*•.¸¸♪